### Las mejoras de enrutamiento de Amazon VPC permiten inspeccionar el tráfico entre subredes en una VPC

#### [Link](https://aws.amazon.com/es/blogs/aws/inspect-subnet-to-subnet-traffic-with-amazon-vpc-more-specific-routing/)

Desde diciembre de 2019, Amazon Virtual Private Cloud (Amazon VPC) le ha permitido enrutar todo el tráfico de entrada (también conocido como tráfico norte - sur) a una interfaz de red específica. Puede utilizar esta capacidad por varias razones. Por ejemplo, para inspeccionar el tráfico entrante mediante un dispositivo de sistema de detección de intrusiones (IDS) o para dirigir el tráfico entrante a un firewall.

Desde que lanzamos esta función, muchos de ustedes nos pidieron que proporcionáramos una capacidad similar para analizar el tráfico que fluye de una subred a otra dentro de su VPC, también conocido como tráfico este-oeste. Hasta hoy, no era posible porque una ruta en una tabla de enrutamiento no puede ser más específica que la ruta local predeterminada (consulte la documentación de la VPC para obtener más detalles). En lenguaje llano, significa que ninguna ruta puede tener un destino que utilice un rango CIDR menor que la ruta local por defecto (que es el rango CIDR de toda la VPC). Por ejemplo, cuando el rango VPC es 10.0.0/16 y una subred tiene 10.0.1.0/24, una ruta a 10.0.1.0/24 es más específica que una ruta a 10.0.0/16.

Las tablas de enrutamiento ya no tienen esta restricción. Las rutas en una tabla de enrutamiento pueden tener rutas más específicas que la ruta local por defecto. Puede utilizar esta ruta más específica para enviar todo el tráfico a un dispositivo o servicio dedicado para inspeccionar, analizar o filtrar todo el tráfico que fluye entre dos subredes (tráfico este-oeste). El destino de la ruta puede ser la interfaz de red (ENI) conectada a un dispositivo que haya construido o adquirido, un punto final de AWS Gateway Load Balancer (GWLB) para distribuir el tráfico a varios dispositivos por motivos de rendimiento o alta disponibilidad, un firewall de red de AWS o una puerta de enlace NAT. También permite insertar un dispositivo entre una subred y un AWS Transit Gateway.

Es posible encadenar appliances para tener más de un tipo de análisis entre subredes de origen y destino. Por ejemplo, es posible que primero desee filtrar el tráfico mediante un firewall (administrado por AWS o un dispositivo de firewall de terceros), en segundo lugar enviar el tráfico a un sistema de detección y prevención de intrusiones y, por último, realizar una inspección profunda de paquetes. Puede acceder a los dispositivos virtuales desde nuestra red de socios de AWS y AWS Marketplace.

Cuando encadenas appliances, cada appliance y cada endpoint tienen que estar en subredes separadas.

In [1]:
import boto3
import json

ec2_r1 = boto3.resource('ec2')
ec2_client_r1 = boto3.client('ec2')

#### Crear una VPC

Para el propósito de este ejemplo vamos a suponer que tengo una VPC con tres subredes. La primera subred es pública y tiene un host bastión. Requiere acceso a recursos, como una API o una base de datos en la segunda subred. La segunda subred es privada. Alberga los recursos que necesita el bastión. Escribí un simple script CDK para ayudarte a desplegar esta configuración.

![image](https://d2908q01vomqb2.cloudfront.net/da4b9237bacccdf19c0760cab7aec4a8359010b0/2021/05/24/illustration-1.png)

In [2]:
vpc_r1 = ec2_r1.create_vpc(CidrBlock='10.10.0.0/16')

In [3]:
# Tag the VPC
vpc_r1.create_tags(Tags=[{"Key": "Name", "Value": "RoutingDemoVPC"}])

[ec2.Tag(resource_id='vpc-03fea7e7026d9f7b7', key='Name', value='RoutingDemoVPC')]

In [4]:
# Crear 3 subredes para la VPC
vpc_subnet_public = ec2_r1.create_subnet(
    CidrBlock='10.10.1.0/24',
    VpcId=vpc_r1.id,
    AvailabilityZone='us-east-1a'
)
vpc_subnet_private_appliance = ec2_r1.create_subnet(
    CidrBlock='10.10.2.0/24', 
    VpcId=vpc_r1.id,
    AvailabilityZone='us-east-1a'
)
vpc_subnet_private_application = ec2_r1.create_subnet(
    CidrBlock='10.10.3.0/24', 
    VpcId=vpc_r1.id,
    AvailabilityZone='us-east-1a'
)

In [5]:
# Create tags
vpc_subnet_public.create_tags(Tags=[{"Key": "Name", "Value": "RoutingDemoVPC-Bastion"}])
vpc_subnet_private_appliance.create_tags(Tags=[{"Key": "Name", "Value": "RoutingDemoVPC-Appliance"}])
vpc_subnet_private_application.create_tags(Tags=[{"Key": "Name", "Value": "RoutingDemoVPC-Application"}])

[ec2.Tag(resource_id='subnet-0efe80915dda30b88', key='Name', value='RoutingDemoVPC-Application')]

In [6]:
# Crear routables para la VPC
vpc_route_table_public = ec2_r1.create_route_table(VpcId=vpc_r1.id)
vpc_route_table_public.create_tags(Tags=[{"Key": "Name", "Value": "RoutingDemoVPC-Public-Bastion"}])
vpc_route_table_appliance = ec2_r1.create_route_table(VpcId=vpc_r1.id)
vpc_route_table_appliance.create_tags(Tags=[{"Key": "Name", "Value": "RoutingDemoVPC-Appliance"}])
vpc_route_table_application = ec2_r1.create_route_table(VpcId=vpc_r1.id)
vpc_route_table_application.create_tags(Tags=[{"Key": "Name", "Value": "RoutingDemoVPC-Application"}])

[ec2.Tag(resource_id='rtb-0aaac0706422ac0e7', key='Name', value='RoutingDemoVPC-Application')]

In [7]:
# Asoociar la tabla de rutas a la subredes
vpc_route_table_public.associate_with_subnet(SubnetId=vpc_subnet_public.id)
vpc_route_table_appliance.associate_with_subnet(SubnetId=vpc_subnet_private_appliance.id)
vpc_route_table_application.associate_with_subnet(SubnetId=vpc_subnet_private_application.id)

ec2.RouteTableAssociation(id='rtbassoc-0156e899eec8ec74f')

In [8]:
# Crear un gateway de Internet (IGW):
internetgateway = ec2_r1.create_internet_gateway(
    TagSpecifications=[{"ResourceType": "internet-gateway", 
    "Tags": [{
            "Key": "Name", 
            "Value": "RoutingDemo-IGW"}
            ]}
    ]
)

In [9]:
# Adjunte el gateway de Internet a la VPC existente:
vpc_r1.attach_internet_gateway(InternetGatewayId=internetgateway.id)

{'ResponseMetadata': {'RequestId': 'b2e40fba-01f4-494f-b8db-5aee2b6d6d4d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b2e40fba-01f4-494f-b8db-5aee2b6d6d4d',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '243',
   'date': 'Wed, 18 Oct 2023 13:48:22 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [10]:
# Cree una ruta de Internet para la tabla de enrutamiento pública:
vpc_route_table_public.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=internetgateway.id
)

ec2.Route(route_table_id='rtb-03ed45c637509b3eb', destination_cidr_block='0.0.0.0/0')

In [11]:
# Crear una Elastic IP address
elastic_ip = ec2_client_r1.allocate_address(Domain='vpc')

In [12]:
# Crear un NAT gateway
natgateway = ec2_client_r1.create_nat_gateway(
    AllocationId=elastic_ip['AllocationId'],
    SubnetId=vpc_subnet_private_application.id
)

In [15]:
# Asociar la tabla de rutas de la subred privada a la puerta de enlace NAT
natgateway_id = natgateway['NatGateway']['NatGatewayId']
vpc_route_table_application.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    NatGatewayId=natgateway_id
)

ec2.Route(route_table_id='rtb-0aaac0706422ac0e7', destination_cidr_block='0.0.0.0/0')

In [16]:
# Crear una interfaz de red (ENI) para la instancia de la aplicación 
network_interface = ec2_client_r1.create_network_interface(
    SubnetId=vpc_subnet_private_appliance.id,
    Description='RoutingDemoVPC-Appliance-ENI'
)

In [18]:
vpc_route_table_application.create_route(
    DestinationCidrBlock='10.10.1.0/24',
    NetworkInterfaceId=network_interface['NetworkInterface']['NetworkInterfaceId']
)

ec2.Route(route_table_id='rtb-0aaac0706422ac0e7', destination_cidr_block='10.10.1.0/24')

In [19]:
vpc_route_table_public.create_route(
    DestinationCidrBlock='10.10.3.0/24',
    NetworkInterfaceId=network_interface['NetworkInterface']['NetworkInterfaceId']
)

ec2.Route(route_table_id='rtb-03ed45c637509b3eb', destination_cidr_block='10.10.3.0/24')

#### Crear 3 instancias de EC2 en cada subred

In [23]:
print("En la terminal ejecute el siguiente comando:")
print(f"    python create_ec2_ssm.py --vpc {vpc_r1.id} --subnet {vpc_subnet_public.id} --tag BastionHost")
print(f"    python create_ec2_ssm.py --vpc {vpc_r1.id} --subnet {vpc_subnet_private_appliance.id} --tag Appliance")
print(f"    python create_ec2_ssm.py --vpc {vpc_r1.id} --subnet {vpc_subnet_private_application.id} --tag Application")

En la terminal ejecute el siguiente comando:
    python create_ec2_ssm.py --vpc vpc-03fea7e7026d9f7b7 --subnet subnet-0e2b8f25d75c0d610 --tag BastionHost
    python create_ec2_ssm.py --vpc vpc-03fea7e7026d9f7b7 --subnet subnet-05d347081705bb23a --tag Appliance
    python create_ec2_ssm.py --vpc vpc-03fea7e7026d9f7b7 --subnet subnet-0efe80915dda30b88 --tag Application


In [ ]:
# Cambie el id de la instancia con el que acaba de crear
account_id = boto3.client('sts').get_caller_identity().get('Account')
instance_id = 'i-0423abb2c223f061c'

In [24]:
vpc_r1

ec2.Vpc(id='vpc-03fea7e7026d9f7b7')